<a href="https://colab.research.google.com/github/gautamHCSCV/AI-ML/blob/main/FBL_object5_mixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import os
from collections import defaultdict
import bisect

In [4]:
z = pd.read_csv('/content/a_inp_z_obj_5.csv')
gk = pd.read_csv('/content/a_out_gk_obj_5.csv')
print(z.head())
gk.head()

   0.85067  0.76295  0.28974        1
0  0.85792  0.67666  0.28126  0.87068
1  0.89523  0.65130  0.26116  0.64944
2  0.93618  0.70254  0.23170  0.42579
3  0.96145  0.68654  0.20020  0.24519
4  0.99155  0.74185  0.17283  0.12884


,3.159,-3.568,0.3805,0.22674,-0.033634,0.029722,-0.012307,0.024285,0.007759,0.012772,-0.0053414
0,3.0080,-3.5635,0.136710,0.41976,-0.053142,0.065635,0.000700,0.036728,-0.006515,0.086254,-0.026632
1,2.7746,-3.3346,-0.061321,0.51460,-0.026471,0.042303,0.019201,0.017243,-0.017620,0.043524,-0.008538
2,2.5506,-2.9690,-0.224540,0.42097,0.040811,-0.040104,0.037589,-0.013955,-0.005813,-0.005594,0.017073
3,2.3522,-2.5314,-0.359090,0.18256,0.124930,-0.078527,0.043146,0.023792,0.001703,0.031520,0.009851
4,2.1882,-2.1056,-0.461560,-0.13631,0.201170,-0.044511,0.022774,0.068382,-0.007223,0.021565,-0.011863


In [9]:
len(z),len(gk.columns)

(77, 11)

In [5]:
from scipy import *
from scipy.linalg import norm, pinv
import math
from sklearn.linear_model import LinearRegression

from matplotlib import pyplot as plt

# y = Wx + P
class RBF:
     
    def __init__(self, indim, outdim, center_inds):
        self.indim = indim
        self.center_inds = center_inds
        self.outdim = outdim
        self.numCenters = len(self.center_inds)
        self.centers = [np.random.uniform(-1, 1, indim) for i in range(self.numCenters)]
        self.W = np.random.random((self.numCenters, self.outdim))
        self.P = LinearRegression()
        self.lr = LinearRegression()
         
    def _basisfunc(self, c, d):
        assert len(d) == self.indim
        return norm(c-d)**1.5
     
    def _calcAct(self, X):
        # calculate activations of RBFs
        G = np.zeros((X.shape[0], self.numCenters), float)
        for ci, c in enumerate(self.centers):
            for xi, x in enumerate(X):
                G[xi,ci] = self._basisfunc(c, x)
        return G
     
    def train(self, X, Y):
        """ X: matrix of dimensions n x indim 
            y: column vector of dimension n x 1 """
         
        # choose random center vectors from training set
        self.P.fit(X,Y)
        rnd_idx = self.center_inds
        self.centers = [X[i,:] for i in rnd_idx]
         
        # print("center", self.centers)
        # calculate activations of RBFs
        G = self._calcAct(X)
        # print(G) 
        # calculate output weights (pseudoinverse)
        self.W = np.dot(pinv(G), Y)
        val = np.zeros((len(X),2))
        val[:,0] = np.dot(G, self.W)
        val[:,1] = self.P.predict(X)
        # print(val.shape)
        self.lr.fit(val,Y)
        
         
    def test(self, X):
        """ X: matrix of dimensions n x indim """
         
        G = self._calcAct(X)
        Y = np.dot(G, self.W)
        Y2 = self.P.predict(X)
        val = np.zeros((len(X),2))
        val[:,0] = Y
        val[:,1] = Y2
        return self.lr.predict(val)

In [7]:
def FBL(value):
    centers_ind = [1]
    got = defaultdict(lambda : 1)
    got[1]=0
    for q in range(11):
        x = np.array(z.iloc[:,:])
        y = np.array(gk.iloc[:,q])
        
        rbf = RBF(4, 1,centers_ind)
        rbf.train(x, y)

        y1 = rbf.test(x)
        maxi,ind = 0,0
        for i in range(len(y)):
                if i not in centers_ind and norm(y[i]-y1[i])/(1+norm(y[i]))>maxi:
                    maxi = norm(y[i]-y1[i])/(1+norm(y[i]))
                    ind = i
        p = 0.5
        error = mean_squared_error(y,y1)**p

        while error>0.15 and len(centers_ind)<len(y)//4:
                centers_ind.append(ind)
                for j in range(ind-3,ind+3):
                    got[j]=0
                rbf = RBF(4, 1, centers_ind)
                rbf.train(x,y)

                y1 = rbf.test(x)

                maxi,ind = 0,0
                for i in range(len(y)):
                    if got[i] and norm(y[i]-y1[i])/(1+norm(y[i]))>maxi:
                        maxi = norm(y[i]-y1[i])/(1+norm(y[i]))
                        ind = i
                error = mean_squared_error(y,y1)**p
                #print(error,ind)
        print(centers_ind)
    return centers_ind

In [8]:
FBL(1)

[1, 11, 24, 38, 65, 49, 16, 3]
[1, 11, 24, 38, 65, 49, 16, 3, 10, 32, 19, 6, 76, 61, 71, 43, 28, 57, 53]
[1, 11, 24, 38, 65, 49, 16, 3, 10, 32, 19, 6, 76, 61, 71, 43, 28, 57, 53]
[1, 11, 24, 38, 65, 49, 16, 3, 10, 32, 19, 6, 76, 61, 71, 43, 28, 57, 53]
[1, 11, 24, 38, 65, 49, 16, 3, 10, 32, 19, 6, 76, 61, 71, 43, 28, 57, 53]
[1, 11, 24, 38, 65, 49, 16, 3, 10, 32, 19, 6, 76, 61, 71, 43, 28, 57, 53]
[1, 11, 24, 38, 65, 49, 16, 3, 10, 32, 19, 6, 76, 61, 71, 43, 28, 57, 53]
[1, 11, 24, 38, 65, 49, 16, 3, 10, 32, 19, 6, 76, 61, 71, 43, 28, 57, 53]
[1, 11, 24, 38, 65, 49, 16, 3, 10, 32, 19, 6, 76, 61, 71, 43, 28, 57, 53]
[1, 11, 24, 38, 65, 49, 16, 3, 10, 32, 19, 6, 76, 61, 71, 43, 28, 57, 53]
[1, 11, 24, 38, 65, 49, 16, 3, 10, 32, 19, 6, 76, 61, 71, 43, 28, 57, 53]


[1, 11, 24, 38, 65, 49, 16, 3, 10, 32, 19, 6, 76, 61, 71, 43, 28, 57, 53]